<font color=red>注：此处是文档第27页</font>

## PyTorch神经网络

#### 神经网络

神经网络可以通过 **torch.nn** 包来构建。

现在对于自动梯度(**autograd**)有一些了解，神经网络是基于自动梯度 (autograd)来定义一些模型。一个 **nn.Module** 包括层和一个方法 **forward(input)** 它会返回输出(**output**)。

一个典型的神经网络训练过程包括以下几点：
1. 定义一个包含可训练参数的神经网络
2. 迭代整个输入
3. 通过神经网络处理输入
4. 计算损失(loss)
5. 反向传播梯度到神经网络的参数
6. 更新网络的参数，典型的用一个简单的更新方法：weight = weight - learning_rate *gradient

In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution(1输入图像通道，6个输出通道，5x5平方卷积)
        # kernel
        self.cov1 = nn.Conv2d(1, 6, 5)
        self.cov2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = W*x + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.cov1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.cov2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension(除批处理尺寸外的所有尺寸)
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net()
print(net)

Net(
  (cov1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (cov2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


你刚定义了一个前馈函数，然后反向传播函数被自动通过 **autograd** 定义了。你可以使用任何张量
操作在前馈函数上。  
一个模型可训练的参数可以通过调用 **net.parameters()** 返回：

In [31]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 5, 5])


让我们尝试随机生成一个 32x32 的输入。注意：期望的输入维度是 32x32 。为了使用这个网络在 **MNIST** 数据集上，你需要把数据集中的图片维度修改为 32x32。

In [32]:
x_input = torch.randn(1, 1, 32, 32)
out = net(x_input)
print(out)

tensor([[ 0.0602, -0.0402, -0.0749,  0.0442,  0.0857, -0.0419,  0.0545,  0.0109,
         -0.0276, -0.0982]], grad_fn=<AddmmBackward>)


把所有参数梯度缓存器置零，用随机的梯度来反向传播

In [33]:
net.zero_grad()
out.backward(torch.randn(1, 10))

在继续之前，让我们复习一下所有见过的类。
- **torch.Tensor**
    - A multi-dimensional array with support for autograd operations like backward(). Also holds the gradient w.r.t. the tensor.
    - 支持autograd操作(如backward())的多维数组。也保持梯度w.r.t张量。
- **nn.Module**
    - Neural network module. Convenient way of encapsulating parameters, with helpers for moving them to GPU, exporting, loading,etc.
    - 神经网络模块。方便的封装参数的方式，有帮助移动他们到GPU，导出，加载等。
- **nn.Parameter** 
    - A kind of Tensor, that is automatically registered as a parameter when assigned as an attribute to a Module.
    - 一种张量，当被指定为模块的属性时，自动作为参数注册。
- **autograd.Function** 
    - Implements forward and backward definitions of an autograd operation. Every Tensor operation, creates at least a single Function node, that connects to functions thatcreated a Tensor and encodes its history.
    - 实现autograd操作的向前和向后定义。每个张量操作，至少创建一个函数节点，连接到创建一个张量并编码其历史的函数。

在此，我们完成了：
1. 定义一个神经网络
2. 处理输入以及调用反向传播

还剩下：
1. 计算损失值
2. 更新网络中的权重

####  损失函数

一个损失函数需要一对输入：模型输出和目标，然后计算一个值来评估输出距离目标有多远。有一些不同的损失函数在 nn 包中。一个简单的损失函数就是 **nn.MSELoss** ，这计算了均方误差。

In [34]:
output = net(x_input)
target = torch.randn(10)  # a dummy target, for example(一个虚构的目标的例子)
target = target.view(1, -1)  # make it the same shape as output（使其与输出拥有相同的shape）
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.0831, grad_fn=<MseLossBackward>)


In [35]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])  # linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

#### 反向传播
为了实现反向传播损失，我们所有需要做的事情仅仅是使用 **loss.backward()**。你**需要清空现存的梯度**，要不然帝都将会和现存的梯度累计到一起。

现在我们调用 loss.backward() ，然后看一下 con1 的偏置项在反向传播之前和之后的变化。

In [36]:
net.zero_grad()  # # zeroes the gradient buffers of all parameters (所有参数的梯度设置为0)

print("conv1.bias.grad before backward")
print(net.cov1.bias.grad)

loss.backward()

print("conv1.bias.grad after backward")
print(net.cov1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0055,  0.0037,  0.0080,  0.0164, -0.0045, -0.0094])


现在我们看到了，如何使用损失函数。  
唯一剩下的事情就是更新神经网络的参数。  
更新神经网络参数：  
最简单的更新规则就是随机梯度下降: weight = weight -learning_rate * gradient

In [37]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

尽管如此，如果你是用神经网络，你想使用不同的更新规则，类似于 **SGD, Nesterov-SGD, Adam，RMSProp**, 等。为了让这可行，我们建立了一个小包：**torch.optim** 实现了所有的方法。使用它非常的简单。

In [38]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.01)

optimizer.zero_grad()  # zero the gradient buffers（梯度缓冲设为0）
output = net(x_input)
loss = criterion(output, target)
loss.backward()
optimizer.step()  # Does the update(是否更新)